In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
%matplotlib inline
import os
    

In [ ]:
# load excel sheets into dataframe
os.listdir('nibrs')
table_list = [pd.read_excel(f'.\\nibrs\\{xlsx}') for xlsx in os.listdir('nibrs')]

In [ ]:
# remove superscript & white spaces from entries
remove_formatting = ["State", "Agency Type", "Agency Name"] # columns we want to target

# apply formatting to each table
for tables in table_list:
    tables[remove_formatting] = (tables[remove_formatting]
    .apply(lambda x: x.str.replace(r'\d+', '', regex=True)) # remove superscript / subscript
    .apply(lambda x: x.str.strip()) # remove white space
    .fillna(method='ffill') # fill in na caused by merged cells
    )

In [ ]:
main_table = table_list[0] 

join_columns = list(main_table.columns[:3]) # columns we want to join on
pattern = '[^(Population)(Agency Type)]' # regex pattern

for tables in table_list[1:]:
    main_table = main_table.merge(tables, on=join_columns, how='outer', suffixes = ['_x', '_y'])    # merge excel sheets

    main_table['Population_x'] = round(np.mean(main_table.filter(regex=r'Population'), axis=1)) # calculate population average
    sum_cols = np.add(                                                                          # add cells based on new 
                        main_table.filter(regex=rf'{pattern}.*_x$').fillna(0), 
                        np.asarray(main_table.filter(regex=rf'{pattern}.*_y$').fillna(0))
                    )
    
    main_table[sum_cols.columns] = sum_cols
    main_table.drop(main_table.filter(regex=rf'_y$'), axis=1, inplace=True)
    main_table.drop_duplicates(subset=join_columns, keep='last', ignore_index=True, inplace=True)
    main_table.columns = main_table.columns.str.rstrip("_x")

In [ ]:
try:
    os.mkdir(f'{os.getcwd()}\\nibrs_output')
except FileExistsError:
    pass


In [ ]:
main_table

In [ ]:

new_columns = ['State', 'Agency Type', 'Agency Name', 'Average Population (2012 - 2020)', 'Total Offenses', 'Crimes Against Persons', 'Crimes Against Property', 'Crimes Against Society']
main_table.columns = new_columns
unique_agency_types = list(main_table['Agency Type'].unique())
unique_agency_types

main_table.to_excel(f'{os.getcwd()}\\nibrs_output\\combined_nibrs_2012-2020.xlsx', na_rep='NA', sheet_name='Cities', header=new_columns, index=False)
